In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext
sqlContext=SQLContext(sc)

## Listado en orden de importancia (del más hablado al menos hablado) de los idiomas que manejan aquellos usuarios que hablan por lo menos tres idiomas

Cargo el CSV y antes de pasarlo a RDD cambio todas las palabras de la columna babel_lang las paso a mayuscula por si hay algun idioma que están escritos en distintas letras que no se diferencien

In [3]:
df_languages=sqlContext.read.csv('languages.csv', header=True)
df_languages = df_languages.select("*", upper(col('babel_lang')))

In [4]:
rdd_languages = df_languages.rdd

Como me pide el listado de importancia, del mas hablado al menos hablado, lo primero que hago es hacer un map de el user_id y el idioma en mayus que creamos en la primer linea como clave y como valor el numero equivalente al idioma hablado, como hay casos en donde el idioma es nativo y se lo represena con un 'N', ese caracter fue reemplazado por el numero 6.
Luego realizo un filtro por todos los usuarios que hablan algun idioma, o sea que su nivel es mayor a cero como se acordo con Damian via Slack. 

In [5]:
rdd_user = rdd_languages.map(lambda x: ((x[0],x[3]),'6' if x[2]=='N' else x[2])).filter(lambda x: x[1] > '0').cache()

Ahora analizo que usuarios hablan por lo menos tres idiomas.

In [6]:
rdd_user_3 = rdd_user.map(lambda x: (x[0][0],1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>3)

Ahora que tengo a todos los usuarios que hablan tres idiomas, voy a armarme un RDD con los ID de los usuarios y con los idiomas que hablan para poder hacer un join

In [7]:
rdd_user_lan = rdd_user.map(lambda x: (x[0][0],x[0][1]))

Hago el Join por ID de usuario que me va a quedar como clave y como valor me queda el idioma y el numero de sumatoria de los idioma que hablan cada user

In [8]:
rdd_join=rdd_user_lan.join(rdd_user_3)

Luego del Join, vuelvo a realizar un map con el idioma y un valor 1 para poder hacer el reduceByKey y sumar la cantidad de veces que está cada idioma en el RDD. Luego lo ordeno de forma descendente y hago un glom para juntar todo en una lista

In [9]:
rdd_list = rdd_join.map(lambda x: (x[1][0],1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: -x[1]).glom()

Hago el collect de la lista para ver en orden de importancia los idiomas mas hablados por los usuarios que hablan mas de tres idiomas

In [10]:
rdd_list.collect()

[[('EN', 3518),
  ('ES', 3452),
  ('FR', 2412),
  ('IT', 1435),
  ('DE', 1273),
  ('PT', 1246),
  ('CA', 923),
  ('LA', 407),
  ('GL', 383),
  ('RU', 341),
  ('JA', 320),
  ('EO', 213),
  ('ZH', 156),
  ('EU', 148),
  ('AR', 143),
  ('NL', 120),
  ('AST', 120),
  ('PL', 111),
  ('SV', 97),
  ('EL', 94),
  ('GRC', 68),
  ('HE', 65),
  ('RO', 61),
  ('UK', 55),
  ('AN', 51),
  ('KO', 45),
  ('NO', 43),
  ('OC', 43),
  ('LAD', 39),
  ('CS', 38),
  ('FI', 37),
  ('DA', 36),
  ('HR', 36),
  ('NAH', 35),
  ('SR', 30),
  ('QU', 30),
  ('TR', 28),
  ('HU', 28),
  ('BG', 28),
  ('GN', 26),
  ('BE', 24),
  ('SK', 23),
  ('FA', 21),
  ('IA', 20),
  ('SL', 19),
  ('GA', 18),
  ('EXT', 17),
  ('ID', 17),
  ('HI', 15),
  ('MK', 15),
  ('ET', 14),
  ('LT', 14),
  ('PT-BR', 14),
  ('BS', 13),
  ('IS', 13),
  ('SA', 12),
  ('SH', 12),
  ('ARN', 12),
  ('QYA', 12),
  ('IO', 11),
  ('SIMPLE', 11),
  ('NN', 11),
  ('TH', 11),
  ('AF', 10),
  ('VEC', 10),
  ('GSW', 10),
  ('SC', 9),
  ('SCN', 9),
  ('NAP',